In [2]:
import pandas as pd
from pathlib import Path

# =====================================================
# 1) Read the results
# =====================================================
# Make sure the file is in your working directory or adjust the path
df = pd.read_csv("../results_summary.csv")

# Inspect structure
print("Loaded shape:", df.shape)
print(df.head())

# =====================================================
# 2) Compute summary statistics per Model (averaged over datasets)
# =====================================================
summary_cols = [
    "Accuracy", "F1_macro", "Train_time_s", "Infer_time_s",
    "CV_Accuracy_mean", "CV_Accuracy_std", "CV_F1_mean", "CV_F1_std"
]

summary = (
    df.groupby("Model", as_index=False)[summary_cols]
      .mean(numeric_only=True)
      .sort_values("Accuracy", ascending=False)
)

# =====================================================
# 3) Count how many datasets each model "won" (highest Accuracy / F1)
# =====================================================
def count_wins(metric):
    winners = df.loc[df.groupby("Dataset")[metric].idxmax()]
    return winners["Model"].value_counts().rename(f"Wins_{metric}")

win_acc = count_wins("Accuracy")
win_f1 = count_wins("F1_macro")

summary = summary.merge(win_acc, left_on="Model", right_index=True, how="left")
summary = summary.merge(win_f1, left_on="Model", right_index=True, how="left")
summary[["Wins_Accuracy", "Wins_F1_macro"]] = summary[["Wins_Accuracy", "Wins_F1_macro"]].fillna(0).astype(int)

# =====================================================
# 4) Create per-dataset pivot tables
# =====================================================
acc_pivot = df.pivot(index="Dataset", columns="Model", values="Accuracy")
f1_pivot = df.pivot(index="Dataset", columns="Model", values="F1_macro")
cv_acc_pivot = df.pivot(index="Dataset", columns="Model", values="CV_Accuracy_mean")
cv_f1_pivot = df.pivot(index="Dataset", columns="Model", values="CV_F1_mean")

# Round values for readability
for d in [summary, acc_pivot, f1_pivot, cv_acc_pivot, cv_f1_pivot]:
    d.round(3)

# =====================================================
# 5) Save outputs (CSV + LaTeX)
# =====================================================
outdir = Path("cross_check_outputs")
outdir.mkdir(exist_ok=True)

summary.to_csv(outdir / "cross_dataset_summary.csv", index=False)
acc_pivot.to_csv(outdir / "per_dataset_accuracy.csv")
f1_pivot.to_csv(outdir / "per_dataset_f1.csv")
cv_acc_pivot.to_csv(outdir / "per_dataset_cv_accuracy.csv")
cv_f1_pivot.to_csv(outdir / "per_dataset_cv_f1.csv")

# Helper to make clean LaTeX tables
def to_latex(df_, caption, label, index=True):
    return df_.to_latex(escape=True, index=index, caption=caption, label=label, float_format="%.3f")

latex_summary = to_latex(summary.set_index("Model"),
                         "Average performance across datasets (higher is better).",
                         "tab:cross_dataset_summary")

latex_acc = to_latex(acc_pivot,
                     "Per-dataset Accuracy by model.",
                     "tab:per_dataset_accuracy")

latex_f1 = to_latex(f1_pivot,
                    "Per-dataset F1-macro by model.",
                    "tab:per_dataset_f1")

latex_cv_acc = to_latex(cv_acc_pivot,
                        "Per-dataset CV Accuracy mean by model.",
                        "tab:per_dataset_cv_accuracy")

latex_cv_f1 = to_latex(cv_f1_pivot,
                       "Per-dataset CV F1 mean by model.",
                       "tab:per_dataset_cv_f1")

# Save all LaTeX tables to one .tex file
with open(outdir / "tables.tex", "w", encoding="utf-8") as f:
    f.write("% Cross-dataset summary\n")
    f.write(latex_summary + "\n\n")
    f.write("% Per-dataset Accuracy\n")
    f.write(latex_acc + "\n\n")
    f.write("% Per-dataset F1_macro\n")
    f.write(latex_f1 + "\n\n")
    f.write("% Per-dataset CV Accuracy\n")
    f.write(latex_cv_acc + "\n\n")
    f.write("% Per-dataset CV F1\n")
    f.write(latex_cv_f1 + "\n\n")

print("✅ Tables and summaries exported to:", outdir.resolve())

# =====================================================
# 6) (Optional) Display in notebook
# =====================================================
try:
    from IPython.display import display, Markdown
    display(Markdown("### Cross-Dataset Summary"))
    display(summary.round(3))
    display(Markdown("### Accuracy per Dataset"))
    display(acc_pivot.round(3))
    display(Markdown("### F1 per Dataset"))
    display(f1_pivot.round(3))
except Exception:
    pass

Loaded shape: (12, 10)
    Dataset         Model  Accuracy  F1_macro  Train_time_s  Infer_time_s  \
0  dataset1       SVM_rbf  0.701299  0.681450      0.150221      0.060483   
1  dataset1  RandomForest  0.896104  0.896240      0.313368      0.021006   
2  dataset1     NeuralNet  0.733766  0.728480      1.310231      0.003317   
3  dataset2       SVM_rbf  0.999724  0.999718    110.288555     17.600029   
4  dataset2  RandomForest  0.999979  0.999978     30.925761      0.375053   

   CV_Accuracy_mean  CV_Accuracy_std  CV_F1_mean  CV_F1_std  
0          0.686249         0.029627    0.670953   0.028535  
1          0.889364         0.028039    0.889636   0.027464  
2          0.731771         0.013212    0.728019   0.014769  
3          0.999763         0.000036    0.999757   0.000037  
4          0.999996         0.000008    0.999996   0.000009  
✅ Tables and summaries exported to: /Users/vp/PycharmProjects/MachineLearning/Practice1/62_CayoPletikosicRavuril_Exercise1_Report/Pipelines/.i

### Cross-Dataset Summary

,Model,Accuracy,F1_macro,Train_time_s,Infer_time_s,CV_Accuracy_mean,CV_Accuracy_std,CV_F1_mean,CV_F1_std,Wins_Accuracy,Wins_F1_macro
1,RandomForest,0.927,0.892,11.978,0.162,0.925,0.009,0.891,0.010,3,3
0,NeuralNet,0.886,0.851,20.585,0.029,0.886,0.004,0.851,0.006,0,1
2,SVM_rbf,0.851,0.808,46.011,7.963,0.855,0.013,0.814,0.014,1,0


### Accuracy per Dataset

Model,NeuralNet,RandomForest,SVM_rbf
Dataset,,,
dataset1,0.734,0.896,0.701
dataset2,1.000,1.000,1.000
dataset3,1.000,1.000,0.885
dataset4,0.810,0.812,0.816


### F1 per Dataset

Model,NeuralNet,RandomForest,SVM_rbf
Dataset,,,
dataset1,0.728,0.896,0.681
dataset2,1.000,1.000,1.000
dataset3,1.000,1.000,0.883
dataset4,0.676,0.671,0.669
